In [65]:
import ipeadatapy as ipea
import pandas as pd

In [78]:
series = ipea.list_series()
series.tail()

,CODE,NAME
9602,ADH_RDPC4_URB,Renda domiciliar per capita média do 4º quint...
9603,ADH_RDPC5_RUR,Renda domiciliar per capita média do quinto ma...
9604,ADH_RDPC5_URB,Renda domiciliar per capita média do quinto ma...
9605,ADH_THEIL_RUR,Índice de Theil - L - áreas rurais
9606,ADH_THEIL_URB,Índice de Theil - L - áreas urbanas


In [89]:
df = (pd.read_csv("deflator.csv"))

In [90]:
bases = {
    "exportacao": ipea.describe("BPN_XTV"),
    "importacao": ipea.describe("BPN_MTV")
}
for base, dados in bases.items():
    bases[base] = list(dados.to_dict().values())[0]

In [91]:
df = df.rename({"DATE": "YEAR", df.columns[1]: "deflator"}, axis = 1).assign(YEAR = lambda _: _.YEAR.str[0:4].astype(int))
for base, dados in bases.items():
    _ = ipea.timeseries(dados["Code"]).reset_index()
    df = pd.merge(
        _.rename({_.columns[6]: base}, axis = 1).filter(["YEAR", base]),
        df, how = "outer", on = "YEAR"
    )
df

,YEAR,importacao,exportacao,deflator
0,1889,116.6,138.8,NaN
1,1890,116.7,128.2,NaN
2,1891,124.2,131.9,NaN
3,1892,127.8,150.0,NaN
4,1893,127.4,155.6,NaN
...,...,...,...,...
129,2018,NaN,NaN,108.317
130,2019,NaN,NaN,109.933
131,2020,NaN,NaN,111.145
132,2021,NaN,NaN,115.621


In [94]:
df.assign(
    importacao = lambda _: _.importacao / _.deflator,
    exportacao = lambda _: _.exportacao / _.deflator,
    # saldo = lambda _: _.saldo / _.deflator
).dropna()

,YEAR,importacao,exportacao,deflator
40,1929,39.554647,49.526678,9.296
41,1930,25.331386,35.879578,8.902
42,1931,14.655932,30.695685,7.949
43,1932,13.238231,25.592011,7.010
44,1933,21.919834,32.066262,6.761
...,...,...,...,...
121,2010,1898.424258,2108.841899,95.747
122,2011,2304.642516,2608.124425,98.170
123,2012,2231.834766,2425.780135,100.000
124,2013,2364.327901,2388.002197,101.354


In [81]:
df_import = pd.read_csv("import.csv")
df_export = pd.read_csv("export.csv", sep = ";", decimal = ",")
deflator = pd.read_csv("deflator.csv")


In [82]:
deflator = deflator.rename({"DATE": "data", deflator.columns[1]: "deflator"}, axis = 1).assign(data = lambda _: _.data.str[0:4].astype(int))
df = pd.merge(
    df_import.rename({"Data": "data", df_import.columns[1]: "importacao"}, axis = 1).filter(["data", "importacao"]),
    df_export.rename({"Data": "data", df_export.columns[1]: "exportacao"}, axis = 1).filter(["data", "exportacao"]),
    how = "outer"
).assign(saldo = lambda _: _.exportacao - _.importacao)

In [83]:
df = df.merge(deflator, how = "inner").assign(
    importacao = lambda _: _.importacao / _.deflator,
    exportacao = lambda _: _.exportacao / _.deflator,
    saldo = lambda _: _.saldo / _.deflator
)

In [74]:
df.to_csv("saldo_deflacionado.csv", index = False)